In [1]:
!pip install surprise

     |████████████████████████████████| 11.8MB 264kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670908 sha256=8677fda689c672eae8f7dc07513027d01edbfc5638519ddbe978be44dac1fca2
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [2]:
!pip install implicit

     |████████████████████████████████| 1.1MB 2.8MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp36-cp36m-linux_x86_64.whl size=3419481 sha256=30e98c856e84bd7599213c62a8c470464506e7e0a7baaadf696e5a6eea6a077d
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


**Import libraries, Read the data¶**


In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import time
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, SlopeOne, NMF, SVDpp, KNNBaseline, CoClustering, accuracy
from surprise.model_selection import cross_validate
from __future__ import (absolute_import, division, print_function,unicode_literals)
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import sys
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random
from sklearn.preprocessing import MinMaxScaler
import implicit


In [5]:
from google.colab import files
uploaded = files.upload()

Saving hotel_data_final.csv to hotel_data_final.csv


In [6]:
import io
df = pd.read_csv(io.BytesIO(uploaded['hotel_data_final.csv']))

In [7]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [8]:
# Explore Data
type(df)
df.size
df.shape
df.head()
df.tail()

pandas.core.frame.DataFrame

280728

(23394, 12)

,Hotel_id,Hotel,Address,City,Title,Users,Reviews,Rating,Hotel_Type,Price,Hotel_Category,User_id
0,0,La Quinta,5820 Walden Rd,Beaumont,This hotel is a dump! Upon ch...,margierodriguez.mcqueen,This hotel is a dump! Upon checking in was tol...,1.0,2.0,Rs.8989,"Lodging,Motels,Hotels,Meeting & Event Planning...",15400
1,1,The Point,Po Box 1327,Saranac Lake,Great family time,motomomMaryland,The point is possibly one of our favorite rest...,5.0,NaN,NaN,"Hotels,Hotel,Restaurants",15740
2,1,The Point,Po Box 1327,Saranac Lake,Throw back to the Gatsby Era - don't miss The ...,ellen00007,From the minute you arrive until the minute yo...,5.0,NaN,NaN,"Hotels,Hotel,Restaurants",13760
3,1,The Point,Po Box 1327,Saranac Lake,The Point of it all,Szerzek,That is the name on one of their boats. And th...,5.0,NaN,NaN,"Hotels,Hotel,Restaurants",11027
4,1,The Point,Po Box 1327,Saranac Lake,The most amazing week end!,Sarahbnyc83,I was lucky enough to spend 2 nights at The Po...,5.0,NaN,NaN,"Hotels,Hotel,Restaurants",10306


,Hotel_id,Hotel,Address,City,Title,Users,Reviews,Rating,Hotel_Type,Price,Hotel_Category,User_id
23389,3128,Polynesian Plaza,2131 Kalakaua Ave,Honolulu,IN SEVERE NEED OF UPDATE - OVERHAUL,fordpipefitter,We spent 2 weeks in this hotel due to the cost...,2.0,2.0,Rs.17718,"Vacation Rentals,Resorts & Vacation Cottages,V...",13911
23390,3129,Quality Inn,2120 Jameson Pl S W,Decatur,NaN,RABBITS,VERY FRIENDLY AND GREAT PRICE. I HAVE STAYED A...,5.0,2.0,Rs.7373,"Lodges,Hotels,Hotels & Motels,Lodging,Motels",9306
23391,3130,Quality Inn,2120 Jameson Pl S W,Decatur,NaN,Contributor,Editorial Review by Citysearch Editors Colonia...,5.0,2.0,Rs.7373,"Lodges,Hotels,Hotels & Motels,Lodging,Motels",2888
23392,3130,Quality Inn,12439 Northwest Fwy,Houston,NaN,Houstontraveler,What Happened to this Place!!!! I come this ho...,NaN,2.0,Rs.7373,"Hotel & Motel Reservations,Hotels,Hotels & Mot...",4949
23393,3130,Quality Inn,12439 Northwest Fwy,Houston,NaN,Robyn Galbos,Budget-minded travelers appreciate amenities l...,NaN,2.0,Rs.7373,"Hotel & Motel Reservations,Hotels,Hotels & Mot...",9807


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23394 entries, 0 to 23393
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Hotel_id        23394 non-null  int64  
 1   Hotel           23394 non-null  object 
 2   Address         23394 non-null  object 
 3   City            23394 non-null  object 
 4   Title           23342 non-null  object 
 5   Users           23394 non-null  object 
 6   Reviews         23393 non-null  object 
 7   Rating          23365 non-null  float64
 8   Hotel_Type      22432 non-null  float64
 9   Price           21881 non-null  object 
 10  Hotel_Category  23394 non-null  object 
 11  User_id         23394 non-null  int64  
dtypes: float64(2), int64(2), object(8)
memory usage: 2.1+ MB


In [10]:
# Checking for missing values
pd.isnull(df).any()
pd.isnull(df).sum()

Hotel_id          False
Hotel             False
Address           False
City              False
Title              True
Users             False
Reviews            True
Rating             True
Hotel_Type         True
Price              True
Hotel_Category    False
User_id           False
dtype: bool

Hotel_id             0
Hotel                0
Address              0
City                 0
Title               52
Users                0
Reviews              1
Rating              29
Hotel_Type         962
Price             1513
Hotel_Category       0
User_id              0
dtype: int64

In [11]:
# Dropping the missing values
df = df.dropna()
df = df.reset_index(drop=True)
print('Shape of dataset ',df.shape)
print(df.columns)

Shape of dataset  (21770, 12)
Index(['Hotel_id', 'Hotel', 'Address', 'City', 'Title', 'Users', 'Reviews',
       'Rating', 'Hotel_Type', 'Price', 'Hotel_Category', 'User_id'],
      dtype='object')


**Train-Test Split**

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(df[['User_id','Hotel_id']],df['Rating'],test_size=0.25, 
                                                  random_state=0)

y_train=pd.DataFrame(y_train)
train=X_train.join(y_train)

y_test=pd.DataFrame(y_test)
test=X_test.join(y_test)

In [13]:
X_train.shape
X_test.shape
y_train.shape
y_test.shape

(16327, 2)

(5443, 2)

(16327, 1)

(5443, 1)

**Collaborative Filtering**

Collaborative Filtering is based on the idea that users similar to a me can be used to predict how much I will like a particular product or service those users have used/experienced but I have not.

**1- Simple Collaborative filtering base on mean**

Recommend based on other user's rating histories.

In [14]:
def collab_mean(user_id, hotel_id):
    user_condition = X_train.Hotel_id == hotel_id
    return y_train[user_condition].mean()

i=30
j=96
print('Collaborative mean filter for userId:',i, 'and hotelid:', j, 'is:',collab_mean(i,j).values)
print('Actual rating value is:', df[(df.User_id==i)&(df.Hotel_id==j)]['Rating'].mean())
print('')

i=3529
j=25
print('Collaborative mean filter for userId:',i, 'and hotelid:', j, 'is:',collab_mean(i,j).values)
print('Actual rating value is:', df[(df.User_id==i)&(df.Hotel_id==j)]['Rating'].mean())
print('')



i=13532
j=9
print('Collaborative mean filter for userId:',i, 'and hotelid:', j, 'is:',collab_mean(i,j).values)
print('Actual rating value is:', df[(df.User_id==i)&(df.Hotel_id==j)]['Rating'].mean())
print('')

Collaborative mean filter for userId: 30 and hotelid: 96 is: [4.18181818]
Actual rating value is: 5.0

Collaborative mean filter for userId: 3529 and hotelid: 25 is: [4.20930233]
Actual rating value is: 4.0

Collaborative mean filter for userId: 13532 and hotelid: 9 is: [4.20430108]
Actual rating value is: 4.0



**2- Matrix Factorization-based algorithms**

The idea is basically to take a large (or potentially huge) matrix and factor it into some smaller representation of the original matrix. You can think of it in the same way as we would take a large number and factor it into two much smaller primes. We end up with two or more lower dimensional matrices whose product equals the original one. When we talk about collaborative filtering for recommender systems we want to solve the problem of our original matrix having millions of different dimensions, but our “tastes” not being nearly as complex. Even if i’ve viewed hundreds of items they might just express a couple of different tastes. Here we can actually use matrix factorization to mathematically reduce the dimensionality of our original “all users by all items” matrix into something much smaller that represents “all items by some taste dimensions” and “all users by some taste dimensions”. These dimensions are called latent or hidden features and we learn them from our data. Doing this reduction and working with fewer dimensions makes it both much more computationally efficient and but also gives us better results since we can reason about items in this more compact “taste space”. If we can express each user as a vector of their taste values, and at the same time express each item as a vector of what tastes they represent. You can see we can quite easily make a recommendation. This also gives us the ability to find connections between users who have no specific items in common but share common tastes.

**SVD**

SVD is a matrix factorization technique that is usually used to reduce the number of features of a data set by reducing space dimensions from N to K where K < N. For the purpose of the recommendation systems however, we are only interested in the matrix factorization part keeping same dimensionality. The matrix factorization is done on the user-item ratings matrix.

I have used Surprise library which has several powerful algorithms like Singular Value Decomposition (SVD), Non-negative Matrix Factorization (NMF), K Nearest Neighbor (KNN), and CoClusternig to minimise RMSE (Root Mean Square Error) and give recommendations.

In [15]:
data = Dataset.load_from_df(df[['User_id', 'Hotel_id', 'Rating']], Reader())


In [16]:
Models=[SVD(), NMF(), SlopeOne(), KNNBaseline(), CoClustering()]

for i in Models:
    start = time.time()
    cross_validate(i, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
    end = time.time()
    print('Total Elapsed time with model is:',end - start)
    print(' ')
    print(' ')

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0788  1.0473  1.0784  1.0866  1.0831  1.0749  0.0141  
MAE (testset)     0.8391  0.8135  0.8340  0.8415  0.8438  0.8344  0.0109  
Fit time          1.05    0.99    1.01    0.99    1.00    1.01    0.02    
Test time         0.09    0.02    0.02    0.02    0.02    0.04    0.03    


{'fit_time': (1.0464560985565186,
  0.9905779361724854,
  1.0107426643371582,
  0.9920146465301514,
  0.9973495006561279),
 'test_mae': array([0.83909614, 0.81348979, 0.83397624, 0.84152725, 0.84378331]),
 'test_rmse': array([1.07884548, 1.04733307, 1.07844404, 1.0865719 , 1.08314563]),
 'test_time': (0.09450316429138184,
  0.02279806137084961,
  0.023642778396606445,
  0.0233151912689209,
  0.02274036407470703)}

Total Elapsed time with model is: 5.461812257766724
 
 
Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2126  1.2044  1.2144  1.2018  1.2138  1.2094  0.0052  
MAE (testset)     0.9414  0.9366  0.9448  0.9334  0.9497  0.9412  0.0058  
Fit time          1.83    1.78    1.77    1.77    1.92    1.81    0.06    
Test time         0.02    0.02    0.02    0.02    0.02    0.02    0.00    


{'fit_time': (1.8292832374572754,
  1.7784719467163086,
  1.7723815441131592,
  1.7682411670684814,
  1.9208838939666748),
 'test_mae': array([0.94140914, 0.93656459, 0.9447658 , 0.93336068, 0.94966533]),
 'test_rmse': array([1.2126287 , 1.20443956, 1.21438188, 1.20182044, 1.21375451]),
 'test_time': (0.02213144302368164,
  0.022569894790649414,
  0.023209333419799805,
  0.02235698699951172,
  0.022383451461791992)}

Total Elapsed time with model is: 9.487185716629028
 
 
Evaluating RMSE, MAE of algorithm SlopeOne on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2149  1.2008  1.1842  1.2040  1.2113  1.2031  0.0107  
MAE (testset)     0.9266  0.9148  0.9072  0.9143  0.9279  0.9182  0.0079  
Fit time          0.26    0.24    0.22    0.24    0.24    0.24    0.01    
Test time         0.04    0.04    0.04    0.04    0.04    0.04    0.00    


...

Total Elapsed time with model is: 1.6524217128753662
 
 
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1029  1.0720  1.0988  1.0867  1.0790  1.0879  0.0116  
MAE (testset)     0.8535  0.8259  0.8552  0.8442  0.8351  0.8428  0.0111  
Fit time          6.82    5.66    5.77    5.25    5.67    5.83    0.52    
Test time         0.10    0.10    0.10    0.10    0.11    0.1

{'fit_time': (6.8166046142578125,
  5.66199517250061,
  5.77459979057312,
  5.25112771987915,
  5.666298151016235),
 'test_mae': array([0.85353305, 0.82594796, 0.85515384, 0.84420913, 0.83506014]),
 'test_rmse': array([1.10292047, 1.07196671, 1.09876316, 1.08673892, 1.0790063 ]),
 'test_time': (0.10165572166442871,
  0.10283541679382324,
  0.10028076171875,
  0.0978238582611084,
  0.10578179359436035)}

Total Elapsed time with model is: 30.009405851364136
 
 
Evaluating RMSE, MAE of algorithm CoClustering on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2118  1.1748  1.1747  1.1786  1.2039  1.1888  0.0159  
MAE (testset)     0.9272  0.8947  0.9025  0.9107  0.9265  0.9123  0.0129  
Fit time          1.35    1.34    1.35    1.32    1.35    1.34    0.01    
Test time         0.02    0.02    0.02    0.02    0.10    0.04    0.03    


{'fit_time': (1.3453168869018555,
  1.3444249629974365,
  1.3460609912872314,
  1.317664384841919,
  1.3463845252990723),
 'test_mae': array([0.92717515, 0.89470537, 0.90253756, 0.91066027, 0.92653346]),
 'test_rmse': array([1.21182167, 1.17481082, 1.17467673, 1.1786495 , 1.20393059]),
 'test_time': (0.019527673721313477,
  0.018418312072753906,
  0.018311500549316406,
  0.02059340476989746,
  0.10477542877197266)}

Total Elapsed time with model is: 7.129789352416992
 
 


**Comparing the predicted rating values of some samples with their actual values**

In [17]:
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Models=[SVD(), NMF(), SlopeOne(), KNNBaseline(), CoClustering()]
m=['SVD','NMF','Slop One','KNN Baseline', 'CoClustering']

count=0

for algo in Models:
    print(m[count])
    start = time.time()
    algo.fit(trainset)
    predictions = algo.test(testset)
    end = time.time()    
    print('Total Elapsed time with model is:',round(end - start,3)) 
    i=30
    j=96
    print('Prediction of a particular user:', algo.predict(i,j, 3).est)
    print('Actual rating value is:', df[(df.User_id==i)&(df.Hotel_id==j)]['Rating'].mean())
    print('')
    i=3529
    j=25
    print('Prediction of a particular user:', algo.predict(i,j, 3).est)
    print('Actual rating value is:', df[(df.User_id==i)&(df.Hotel_id==j)]['Rating'].mean())
    print('')
    i=13532
    j=9
    print('Prediction of a particular user:', algo.predict(i,j, 3).est)
    print('Actual rating value is:', df[(df.User_id==i)&(df.Hotel_id==j)]['Rating'].mean())
    print('')
    print('')
    count+=1

SVD


Total Elapsed time with model is: 1.012
Prediction of a particular user: 4.164908560249461
Actual rating value is: 5.0

Prediction of a particular user: 4.028169025167626
Actual rating value is: 4.0

Prediction of a particular user: 4.091309840361914
Actual rating value is: 4.0


NMF


Total Elapsed time with model is: 1.752
Prediction of a particular user: 4.925624044390267
Actual rating value is: 5.0

Prediction of a particular user: 3.85745029798112
Actual rating value is: 4.0

Prediction of a particular user: 3.938733832215317
Actual rating value is: 4.0


Slop One


Total Elapsed time with model is: 0.261
Prediction of a particular user: 5
Actual rating value is: 5.0

Prediction of a particular user: 3.606837606837607
Actual rating value is: 4.0

Prediction of a particular user: 4.0
Actual rating value is: 4.0


KNN Baseline
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


Total Elapsed time with model is: 5.038
Prediction of a particular user: 4.994850162108351
Actual rating value is: 5.0

Prediction of a particular user: 4.172513791012143
Actual rating value is: 4.0

Prediction of a particular user: 4.014252512846726
Actual rating value is: 4.0


CoClustering


Total Elapsed time with model is: 1.317
Prediction of a particular user: 4.9904877195158726
Actual rating value is: 5.0

Prediction of a particular user: 2.567083464196722
Actual rating value is: 4.0

Prediction of a particular user: 4.070300906639382
Actual rating value is: 4.0




**3- Alternating Least Squares (ALS) Collaborative Filtering**

Alternating Least Squares (ALS) is a the model we’ll use to fit our data and find similarities. ALS is an iterative optimization process where we for every iteration try to arrive closer and closer to a factorized representation of our original data. We have our original matrix R of size u x i with our users, items and some type of feedback data. We then want to find a way to turn that into one matrix with users and hidden features of size u x f and one with items and hidden features of size f x i. In U and V we have weights for how each user/item relates to each feature. What we do is we calculate U and V so that their product approximates R as closely as possible: R ≈ U x V.

By randomly assigning the values in U and V and using least squares iteratively we can arrive at what weights yield the best approximation of R. The least squares approach in it’s basic forms means fitting some line to the data, measuring the sum of squared distances from all points to the line and trying to get an optimal fit by minimising this value.

With the alternating least squares approach we use the same idea but iteratively alternate between optimizing U and fixing V and vice versa. We do this for each iteration to arrive closer to R = U x V.

In [18]:
data=df
sparse_item_user = sparse.csr_matrix((data['Rating'].astype(float), (data['Hotel_id'], data['User_id'])))
sparse_user_item = sparse.csr_matrix((data['Rating'].astype(float), (data['Hotel_id'], data['User_id'])))

# Initialize the als model and fit it using the sparse item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)

# Calculate the confidence by multiplying it by our alpha value.
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')

model.fit(data_conf)

GPU training requires factor size to be a multiple of 32. Increasing factors from 20 to 32.


**Finding Hotels Similar to Best Western Lamplighter Inn Suites at SDSU:**

In [19]:
#---------------------
# FIND SIMILAR HOTELS
#---------------------
Hotel_id = 318 #Best Western Lamplighter Inn Suites at SDSU
n_similar = 10

# Use implicit to get similar items.
similar = model.similar_items(Hotel_id, n_similar)

a=[]
b=[]
c=[]
# Print the names of our most similar movies
for item in similar:
    idx, score = item
    a.append(data.Hotel_id.loc[data.Hotel_id == idx].iloc[0])
    b.append(score)
    recommendations = pd.DataFrame({'HotelId': a, 'Score': b})

for i in a:
    c.append(df[df.Hotel_id==i].Hotel.iloc[0])
recommendations['Hotel']=c
recommendations

,HotelId,Score,Hotel
0,318,1.000000,Best Western Lamplighter Inn Suites at SDSU
1,1049,0.999586,Hampton Inn Myrtle Beach-Northwood
2,1045,0.840715,Courtyard Macon
3,2292,0.806033,Home2 Suites By Hilton Greenville Airport
4,2894,0.775714,Americas Best Value Inn
5,431,0.642421,Hilton Garden Inn Atlanta Midtown
6,2880,0.617103,Best Western Plus St Charles Inn
7,2322,0.614468,Econo Lodge Inn & Suites Carrollton Smithfield
8,1780,0.561289,Comfort Suites Alamo/River walk
9,2796,0.540337,Sleep Inn And Suites Danville


**Finding Hotels Similar to Nob Hill Hotel:**

In [20]:
#---------------------
# FIND SIMILAR HOTELS
#---------------------
Hotel_id = 296 #Nob Hill Hotel
n_similar = 10

# Use implicit to get similar items.
similar = model.similar_items(Hotel_id, n_similar)

a=[]
b=[]
c=[]
# Print the names of our most similar movies
for item in similar:
    idx, score = item
    a.append(data.Hotel_id.loc[data.Hotel_id == idx].iloc[0])
    b.append(score)
    recommendations = pd.DataFrame({'HotelId': a, 'Score': b})

for i in a:
    c.append(df[df.Hotel_id==i].Hotel.iloc[0])

recommendations['Hotel']=c
recommendations

,HotelId,Score,Hotel
0,296,1.000000,Nob Hill Hotel
1,713,0.975761,Barn Motor Inn
2,1291,0.975618,Magnuson Grand Hotel
3,904,0.653636,Best Western Plus Newark/christiana Inn
4,2562,0.651848,Millennium Hotel Durham
5,2107,0.617364,Rodeway Inn
6,2937,0.605442,Hilton Garden Inn San Diego Rancho Bernardo
7,1678,0.553272,Ramada Louisville Expo Center
8,1207,0.538107,Algoma Beach Motel
9,1695,0.533104,Meadowlands Plaza Hotel


**Recommend Hotels to User_id = 1:**

In [21]:
#------------------------------
# CREATE USER RECOMMENDATIONS
#------------------------------

# Create recommendations for user with id 1
User_id =1

# Use the implicit recommender.
recommended = model.recommend(User_id, sparse_user_item)

hotels = []
scores = []
c=[]

for item in recommended:
    idx, score=item
    hotels.append(data.Hotel_id.loc[data.Hotel_id==idx].iloc[0])
    scores.append(score)


recommendations = pd.DataFrame({'HotelId': hotels, 'Score': scores})

for i in hotels:
    c.append(df[df.Hotel_id==i].Hotel.iloc[0])
recommendations['Hotel']=c
recommendations

,HotelId,Score,Hotel
0,3099,0.665178,Shangri-La Hotel Bengaluru
1,3036,0.541892,Conrad Bengaluru
2,800,0.541709,Fairfield Inn Philadelphia Airport
3,3070,0.526297,Taj West End
4,2673,0.456126,Best Western Plus Inn At The Vines
5,3068,0.438973,The Park Bangalore
6,3072,0.433528,"The Oberoi, Bengaluru"
7,3084,0.432478,The Lalit Ashok
8,323,0.397878,Best Western Plus Pavilions
9,3081,0.397613,Hotel Royal Orchid Regenta Bangalore


**Recommend Hotels to User_id = 75:**

In [22]:
#------------------------------
# CREATE USER RECOMMENDATIONS
#------------------------------

# Create recommendations for user with id 75
User_id =75

# Use the implicit recommender.
recommended = model.recommend(User_id, sparse_user_item)

hotels = []
scores = []
c=[]

for item in recommended:
    idx, score=item
    hotels.append(data.Hotel_id.loc[data.Hotel_id==idx].iloc[0])
    scores.append(score)


recommendations = pd.DataFrame({'HotelId': hotels, 'Score': scores})

for i in hotels:
    c.append(df[df.Hotel_id==i].Hotel.iloc[0])
    
recommendations['Hotel']=c
recommendations

,HotelId,Score,Hotel
0,90,0.893201,Orchid Key Inn
1,1300,0.319781,Best Western Seven Seas
2,2684,0.311620,"Mandarin Oriental, Miami"
3,1143,0.279342,River Hotel
4,92,0.276830,Holiday Inn Express New York City Times Square
5,47,0.273765,Best Western Mission Bay
6,93,0.270646,Hotel Abri
7,1549,0.270443,Stone Brook Resort
8,7,0.269845,Residence Inn By Marriott Irvine John Wayne Ai...
9,2955,0.264512,Chicago Marriott O'Hare
